# Song Lyrics → One-Sentence Summary
Get the lyrics of a song and summarize its main idea in about one sentence.

## Setup
Import required libraries: environment vars, display helper, OpenAI client, BeautifulSoup, and requests.

In [ ]:
import os
from IPython.display import Markdown, display
from openai import OpenAI
from bs4 import BeautifulSoup
import requests

## Function: Get Lyrics from Genius
Fetch and extract the lyrics from a Genius.com song page using BeautifulSoup.

In [ ]:
def get_lyrics_from_genius(url: str) -> str:
    """
    Extracts song lyrics from a Genius.com song URL using BeautifulSoup.
    Example URL: https://genius.com/Ed-sheeran-shape-of-you-lyrics
    """
    # Standard headers to fetch a website
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # raises error if page not found

    soup = BeautifulSoup(response.text, "html.parser")

    # Genius stores lyrics inside <div data-lyrics-container="true">
    lyrics_blocks = soup.find_all("div", {"data-lyrics-container": "true"})

    if not lyrics_blocks:
        return "Lyrics not found."

    # Join all text blocks and clean up spacing
    lyrics = "\n".join(block.get_text(separator="\n") for block in lyrics_blocks)
    return lyrics.strip()

## Function: Create Genius URL
Build a Genius.com lyrics URL automatically from the given artist and song name.

In [ ]:
def create_genius_url(artist: str, song: str) -> str:
    """
    Creates a Genius.com lyrics URL from artist and song name.
    Example:
        create_genius_url("Ed sheeran", "shape of you")
        → https://genius.com/Ed-sheeran-shape-of-you-lyrics
    """
    artist = artist.strip().replace(" ", "-")
    song = song.strip().replace(" ", "-")
    return f"https://genius.com/{artist}-{song}-lyrics"


## Generate URL and Fetch Lyrics
Create the Genius URL from the artist and song name, then fetch and display the lyrics.

In [ ]:
artist = "Ed sheeran"
song = "shape of you"

url = create_genius_url(artist, song)
print(url)
# Output: https://genius.com/Ed-sheeran-shape-of-you-lyrics

user_prompt = get_lyrics_from_genius(url)
print(user_prompt[:5000]) 

In [ ]:
system_prompt = """
You are a **helpful assistant** that specializes in analyzing **song lyrics**.

## Task
Your goal is to **summarize the main idea or theme of a song** in **about one sentence**.

## Instructions
1. Read the given song lyrics carefully.
2. Identify the **core message**, **emotion**, or **story** of the song.
3. Respond with **one concise sentence** only.
4. The tone of your summary should reflect the song’s mood (e.g., joyful, melancholic, romantic, rebellious).

## Edge Cases
- **Very short lyrics:** Summarize the implied meaning.
- **Repetitive lyrics:** Focus on the message or emotion being emphasized.
- **Abstract or nonsensical lyrics:** Describe the overall feeling or imagery they create.
- **No lyrics or only a title provided:** Reply with  
  `No lyrics provided — unable to summarize meaningfully.`
- **Non-English lyrics:** Summarize in English unless otherwise instructed.

## Output Format
Plain text — a single, coherent sentence summarizing the main idea of the song.
"""

## Create Chat Messages
Prepare the system and user messages, then send them to the OpenAI model for summarization.

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model = "gpt-4.1-mini",
    messages = messages
)

## Display Summary
Show the model’s one-sentence summary of the song lyrics in a formatted Markdown output.

In [ ]:
display(Markdown(response.choices[0].message.content))